In [19]:
import random
from music21 import note, stream, converter, environment
from PIL import Image
import os

In [20]:
# Define the range of pitches and note durations
pitches = [
    'C3', 'C#3', 'Db3', 'D3', 'D#3', 'Eb3', 'E3', 'F3', 'F#3', 'Gb3', 'G3', 'G#3', 'Ab3', 'A3', 'A#3', 'Bb3', 'B3',
    'C4', 'C#4', 'Db4', 'D4', 'D#4', 'Eb4', 'E4', 'F4', 'F#4', 'Gb4', 'G4', 'G#4', 'Ab4', 'A4', 'A#4', 'Bb4', 'B4',
    'C5', 'C#5', 'Db5', 'D5', 'D#5', 'Eb5', 'E5'
]
durations = ['whole', 'half', 'quarter', 'eighth', '16th']


In [21]:
os.makedirs('note_images', exist_ok=True)

# Set the MuseScore executable path
us = environment.UserSettings()
us['musicxmlPath'] = '/Applications/MuseScore 4.app/Contents/MacOS/mscore'
us['musescoreDirectPNGPath'] = '/Applications/MuseScore 4.app/Contents/MacOS/mscore'

In [22]:
def create_image(pitch, duration, save_path):
    n = note.Note(pitch, type = duration)
    s = stream.Stream([n])
    s.write('musicxml.png', fp=save_path)

In [23]:
combs = [(pitch, duration) for pitch in pitches for duration in durations]
random.shuffle(combs)

In [25]:
for i, (pitch, duration) in enumerate(combs):
    save_path = f'note_images/{i}_{pitch}_{duration}.png'
    create_image(pitch, duration, save_path)

    img = Image.open(save_path)
    img = Image.resize((128,128))
    img.save(save_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ninjamac/code/urtrapped247/Consonance/Notebooks/note_images/0_F#3_eighth.png'

# Create a way to generate music

## Generate Synthetic MusicXML

In [22]:
from music21 import stream, note, duration, pitch
import os
import random


def generate_random_note():
    # Define a list of possible pitches and durations
    pitches = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5']
    durations = ['whole', 'half', 'quarter', 'eighth', '16th']

    # Select a random pitch and duration
    selected_pitch = random.choice(pitches)
    selected_duration = random.choice(durations)

    # Create and return a music21 note
    n = note.Note()
    n.pitch = pitch.Pitch(selected_pitch)
    n.duration = duration.Duration(selected_duration)
    return n


def generate_synthetic_musicxml(num_samples=15, output_folder='../data/generated_music'):
    # check for output - Use this for .py
        # output_folder = os.path.join(os.path.dirname(__file__), os.pardir, 'musicxml_files')
        # if not os.path.exists(output_folder):
        #     os.makedirs(output_folder)

    #for our notebook since __file__ seems to not work
    current_dir = os.getcwd()
    output_folder = os.path.abspath(os.path.join(current_dir, output_folder))
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    synthetic_data = []

    for i in range(num_samples):
        s = stream.Stream()
        for _ in range(20):  # Generate 20 random notes per sheet
            n = generate_random_note()
            s.append(n)
        s.write('musicxml', fp=f'{output_folder}/sheet_{i}.musicxml')
        synthetic_data.append(s)

    return synthetic_data

In [23]:
synthetic_data = generate_synthetic_musicxml()


## Convert the XML score to a PNG image file


In [28]:
import subprocess
import platform
import os


# get the right path for musescore based on system
def get_musescore_path():
    system = platform.system()
    if system == 'Windows':
        return r'C:\Program Files\MuseScore 4\bin\MuseScore4.exe'  # Update this path if necessary
    elif system == 'Darwin':  # macOS
        return '/Applications/MuseScore 4.app/Contents/MacOS/mscore'
    elif system == 'Linux':
        return '/usr/bin/musescore4'  # Update this path if necessary
    else:
        raise ValueError("Unsupported operating system")


def convert_musicxml_to_png(input_folder='../data/generated_music', output_folder='../data/sheet_images'):
    current_dir = os.getcwd()
    input_folder = os.path.abspath(os.path.join(current_dir, input_folder))
    output_folder = os.path.abspath(os.path.join(current_dir, output_folder))
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    musescore_path = get_musescore_path()

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.musicxml'):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name.replace('.musicxml', '.png'))
            result = subprocess.run([musescore_path, input_path, '-o', output_path], stderr=subprocess.PIPE)
            if result.returncode != 0:
                # Handle or log the error if needed
                print(f"Error processing {file_name}: {result.stderr.decode('utf-8')}")

    return None

In [ ]:
convert_musicxml_to_png()

## Generate Labels for the Y 


In [29]:
import json
import os

def generate_labels(synthetic_data, output_folder='../data/labels'):
    current_dir = os.getcwd()
    output_folder = os.path.abspath(os.path.join(current_dir, output_folder))
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, s in enumerate(synthetic_data):
        labels = {"symbols": []}
        for element in s.notes:
            labels["symbols"].append({
                "type": element.duration.type,
                "pitch": element.pitch.nameWithOctave,
                "position": [element.offset, 0]  # Simplified position
            })
        with open(f'{output_folder}/label_{i}.json', 'w') as f:
            json.dump(labels, f, indent=4)


In [30]:
# Generate labels for the synthetic data
generate_labels(synthetic_data)